In [3]:
!pip install python-dotenv



In [4]:
from dotenv import load_dotenv, dotenv_values
import os

load_dotenv("../key.env")
jinaai_api_key = os.getenv("JINAAI_API_KEY")

#print(os.getenv("JINAAI_API_KEY"))


In [5]:
from llama_index.embeddings.jinaai import JinaEmbedding

text_embed_model = JinaEmbedding(
    api_key=jinaai_api_key,
    model="jina-embeddings-v3",
    # choose `retrieval.passage` to get passage embeddings
    task="retrieval.passage",
)



In [6]:
gesetze = ['../Gesetze/AGG.txt','../Gesetze/ArbSchG.txt','../Gesetze/ArbzG.txt','../Gesetze/BetrVG.txt',
           '../Gesetze/BUrlG.txt','../Gesetze/bgb_611_630.txt','../Gesetze/entgfg.txt','../Gesetze/KSchG.txt',
           '../Gesetze/MiLoG.txt','../Gesetze/NachwG.txt','../Gesetze/TzBfG.txt'
           ]


In [7]:
documents = []
for gesetz in gesetze:
    with open(gesetz, 'r', encoding='utf-8') as f:
        documents.append(f.read())
        print(gesetz)


../Gesetze/AGG.txt
../Gesetze/ArbSchG.txt
../Gesetze/ArbzG.txt
../Gesetze/BetrVG.txt
../Gesetze/BUrlG.txt
../Gesetze/bgb_611_630.txt
../Gesetze/entgfg.txt
../Gesetze/KSchG.txt
../Gesetze/MiLoG.txt
../Gesetze/NachwG.txt
../Gesetze/TzBfG.txt


In [8]:
def chunk_text(text, chunk_size=4000, overlap=400):
    """
    Teilt Text in Chunks mit definierter Größe und Überlappung auf.
    chunk_size: Anzahl Zeichen pro Chunk (4000 ≈ 1000 Tokens)
    overlap: Anzahl überlappender Zeichen zwischen Chunks
    """
    chunks = []
    start = 0
    text_length = len(text)
    
    while start < text_length:
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += (chunk_size - overlap)
    
    return chunks

# 3. Alle Dokumente chunken


In [9]:
# document_names erstellen (vor der Schleife!)
document_names = []
for gesetz in gesetze:
    document_names.append(gesetz.split('/')[-1].replace('.txt', ''))

# Jetzt chunken
all_chunks = []
chunk_metadata = []

for idx, doc in enumerate(documents):
    doc_chunks = chunk_text(doc, chunk_size=4000, overlap=400)
    
    # Chunks sammeln mit Metadaten
    for chunk_idx, chunk in enumerate(doc_chunks):
        all_chunks.append(chunk)
        chunk_metadata.append({
            'gesetz': document_names[idx],
            'chunk_id': chunk_idx,
            'total_chunks': len(doc_chunks)
        })

In [10]:
embeddings = text_embed_model.get_text_embedding_batch(all_chunks)

print(len(embeddings))
print(embeddings[0][:5])

125
[0.00615968, -0.11915848, 0.05040364, 0.00879177, -0.05993237]


In [11]:
import chromadb

from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from IPython.display import Markdown, display

In [12]:
chroma_client = chromadb.PersistentClient(path="../VektorDatenBank_Gesetze")

chroma_collection = chroma_client.get_or_create_collection("VektorDatenBank_Gesetze",
              metadata={"description": "Deutsche Arbeitsgesetze Embeddings"}
)




In [16]:
ids = [f"chunk_{i}" for i in range(len(all_chunks))]

chroma_collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=all_chunks,    
    metadatas=chunk_metadata  
)
print(chroma_collection)

Collection(name=VektorDatenBank_Gesetze)


In [14]:
from llama_index.core import StorageContext


vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
